In [1]:
# 為了美觀 我把Future Warning關閉
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
import gdown
output = "pttsmalldata.zip"
id = "1BT4h4-kzrtCS_52P2i7C1rlj1GZgEbs6"
gdown.download(id=id, output=output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1BT4h4-kzrtCS_52P2i7C1rlj1GZgEbs6
To: /content/pttsmalldata.zip
100%|██████████| 49.2M/49.2M [00:00<00:00, 145MB/s]


'pttsmalldata.zip'

In [6]:
# 把 zip 解壓縮到 data 資料夾
import zipfile
import os
if not os.path.exists("data"):
    f = zipfile.ZipFile("./pttsmalldata.zip")
    f.extractall("./data")
    f.close()

In [7]:
import glob
import json
import pandas as pd
# 拿取所有 *.json 檔案
fs = glob.glob("./data/pttsmalldata/*.json")
# 我們只取用 內文 標題 和 型態(問卦...等等)
# 但實際上我們只使用 內文 做訓練而已
contents = []
titles = []
types = []
for fn in fs:
    with open(fn, "r", encoding="utf-8") as f:
        data = json.load(f)
        contents.append(data["post_content"])
        titles.append(data["post_title"])
        types.append(data["post_type"])
df = pd.DataFrame({
    "type":types,
    "title":titles,
    "content":contents
}, columns=["type", "title", "content"])
df

,type,title,content
0,問卦,看過除暴五恐龍的都幾歲了？,\n除暴五恐龍\n\n一部很勵志人心的卡通\n\n應該沒什麼人看過吧？\n\n看過的都幾歲了...
1,新聞,最新民調「郭柯、柯郭」聯盟皆勝英韓 林,\n1.媒體來源:\n\n風傳媒\n\n2.記者署名\n\n許玲瑄\n\n3.完整新聞標題:...
2,問卦,我在香港，沒錢了怎麼辦,\n我在香港，港幣帶不夠\n請問有人現在也在香港的嗎？\n我想要港幣1000，面交\n我可以...
3,問卦,歐美怎麼不用體罰就能教出有教養的小孩？,\n華人小孩就是欠打 玩具不收好就是打到你收\n\n在外面鬼吼鬼叫就是打 打到你不敢叫\n\...
4,問卦,上班不要看-爆尿公社鬼月篇的真實性？,\nhttps://youtu.be/AnzqRNkymvg\n\n前幾天看完後 一直有個疑...
...,...,...,...
19995,問卦,獅子座男是不是爆幹扯 超自我感覺良好?,\n小弟周遭朋友同學\n\n獅子座男性真的讓人受不了\n\n某案例 一個8+9沒背景沒老大靠...
19996,新聞,大賺2020選票？國民黨：民進黨不要操弄,\n今日新聞\n\n大賺2020選票？國民黨：民進黨不要操弄香港反送中！\n\n記者 陳弘志...
19997,問卦,刮完鬍子 帥度爆表 o'_'o,\n忙到一個禮拜沒刮鬍子了\n看著鏡子裡那個滿嘴噁爛鬍子的我\n感覺就像落魄的浪子\n\n如...
19998,問卦,有沒有喇叭敏感症的八卦,\n\n\n是說啊\n\n台灣人真的都不能忍受被按喇叭欸\n\n一下就逼車、砸車、打架\n\...


In [8]:
# 由於訓練會花超級久時間, 挑選前十篇進行訓練
df = df.head(10)

In [9]:
# 使用 jieba 進行分詞
import jieba
import os
from urllib.request import urlretrieve

# 使用大型字典
big_dict_path = "dict.txt.big"
if not os.path.exists(big_dict_path):
    print("下載大型字典")
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, big_dict_path)
jieba.set_dictionary(big_dict_path)

# 需加入一些鄉民常用字彙
ptt_dict_path = "ptt_dic.txt"
if os.path.exists(ptt_dict_path):
    print("載入ptt專用詞典")
    jieba.load_userdict(ptt_dict_path)

下載大型字典


In [10]:
# 將標點符號去掉
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')


In [11]:
import re
content = df.iloc[0]["content"]
# 去掉網址 ptt的文章內容基本上都會換行 我們順便把最後的換行字元去掉
content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
# 使用 filter 去掉標點符號
content = " ".join(filter(lambda x: x not in punct, jieba.cut(content)))
# 去掉換行符號
content = content.replace("\n", "").replace("\r", "")
content

Building prefix dict from /content/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.267 seconds.
DEBUG:jieba:Loading model cost 1.267 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


' 除暴 五 恐龍   一部 很 勵志 人心 的 卡通   應該 沒什麼 人 看過 吧   看過 的 都 幾歲 了 啊   有沒有 卦   -- '

In [12]:
# 對表格的每一筆都做出轉換
def process(content):
    content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
    content = "".join(filter(lambda x: x not in punct, content))
    content = (" ".join(jieba.cut(content))
                  .replace("\n", "")
                  .replace("\r", ""))
    return content
content_cut = df["content"].apply(process)
content_cut

0     除暴 五 恐龍   一部 很 勵志 人心 的 卡通   應該 沒什麼 人 看過 吧   看...
1     1 媒體 來源   風傳 媒   2 記者 署名   許玲 瑄   3 完整 新聞標題  ...
2     我 在 香港 港幣 帶 不夠  請問 有人 現在 也 在 香港 的 嗎  我 想要 港幣 ...
3     華人 小孩 就是 欠 打 玩具 不收 好 就是 打到 你 收   在 外面 鬼 吼 鬼 叫...
4     前 幾天 看 完 後 一直 有個 疑問  小歐 感覺 到 有人 之後  在 3023 跟 ...
5      之前 很多 人 很 喜歡 用  2014 年柯 O 哲 的話 去 打 臉 2019 年 ...
6      小弟 慶幸 活在 這個 開心 的 城市   買 了 個 炸雞腿 便當   才 60 元 ...
7     首先 哈哈 我 比 上面 那個 還多 05K   了不起 吧    那麼 問題 來 了 大...
8     ※ 發文 無 16 小 標格 式 或 未 依 順序 任意 刪除 者 會 被刪 文   1 ...
9     為 什麼 8 萬 包養 沒事 5k 買春 犯法   你 可以 花 幾萬 包養 女 大生 目...
Name: content, dtype: object

In [24]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
import gensim

In [16]:
# 太少字的句子我們去掉, 因為這樣無法移動
corpus = [sentence for sentence in content_cut if sentence.count(' ') >= 2]
# 將每一個詞轉成數字
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
# 統計訓練資料有多少個詞
nb_samples = sum(len(s) for s in corpus)
# 統計我們有多少種詞
V = len(tokenizer.word_index) + 1
# 降維成100
dim = 100
# 上文取2 下文取2
window_size = 2

In [25]:
def generate_data(corpus, window_size, V):
    # 拿最多 上文(2) + 下文(2)
    maxlen = window_size * 2
    # 拿出每一句
    for words in corpus:
        L = len(words)
        # 拿出每一個單字
        for index, word in enumerate(words):
            # contexts: [上文, 下文]
            # labels: 目標
            contexts = []
            labels   = []
            # 上文的最前面座號
            s = index - window_size
            # 下文的最後面座號
            e = index + window_size
            # range記得要多加1
            contexts.append([words[i] for i in range(s, e + 1)
                                      if 0 <= i < L and i != index])
            labels.append(word)
            # 少於四個的前面補上0
            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            # 答案做出one-hot encoding
            y = to_categorical(labels, V)
            yield (x, y)

In [21]:
cbow = Sequential()
# 使用嵌入層來得到轉化過的語意
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
# 針對4個詞的100語意做平均
cbow.add(GlobalAveragePooling1D())
# 輸出判斷
cbow.add(Dense(V, activation='softmax'))
cbow.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 4, 100)            70000     
                                                                 
 global_average_pooling1d_1  (None, 100)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_1 (Dense)             (None, 700)               70700     
                                                                 
Total params: 140700 (549.61 KB)
Trainable params: 140700 (549.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
cbow.compile(loss='categorical_crossentropy', optimizer='adam')

In [31]:
# 訓練十個epochs
for i in range(10):
    loss = 0.
    for x, y in generate_data(corpus, window_size, V):
        loss += cbow.train_on_batch(x, y)
    print("-" * 15, "Iteration", i, "-" * 15)
    print("loss:", loss)

--------------- Iteration 0 ---------------
loss: 1452.5961354095489
--------------- Iteration 1 ---------------
loss: 1076.7466220017523
--------------- Iteration 2 ---------------
loss: 787.6164875980467
--------------- Iteration 3 ---------------
loss: 571.2596975909546
--------------- Iteration 4 ---------------
loss: 412.28722441755235
--------------- Iteration 5 ---------------
loss: 296.75339055480435
--------------- Iteration 6 ---------------
loss: 213.38747654738836
--------------- Iteration 7 ---------------
loss: 153.52714998461306
--------------- Iteration 8 ---------------
loss: 110.6750783023308
--------------- Iteration 9 ---------------
loss: 80.03276604198618


In [27]:
# 準備寫入我們的向量
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

8

In [28]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    # 存擋的時候必須是 2 3 1 這樣空白鍵在中間
    # 但是在用join時裡面必須是字串, 所以先用map轉換成字串
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [29]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [33]:
w2v.most_similar("女生")

[('皮包', 0.8038417100906372),
 ('阿', 0.716313362121582),
 ('放', 0.6347060799598694),
 ('60007000', 0.5888465046882629),
 ('名字', 0.5794138312339783),
 ('錢', 0.5374957323074341),
 ('沒有', 0.49924588203430176),
 ('變法', 0.43230631947517395),
 ('最後', 0.42194804549217224),
 ('女', 0.40628981590270996)]

In [34]:
w2v.most_similar("台北")

[('市長', 0.6259551048278809),
 ('內文', 0.5991846323013306),
 ('高雄', 0.5583660006523132),
 ('連結', 0.5555239319801331),
 ('和', 0.5288509726524353),
 ('哲為', 0.5213826894760132),
 ('哲與', 0.5167677998542786),
 ('哲', 0.49126845598220825),
 ('瑜', 0.4685736894607544),
 ('韓國', 0.467721164226532)]